In [102]:
import mdptoolbox
import numpy as np

isBad = [1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0]
prob = 1/len(isBad)
rolls = 6
nStates = rolls*len(isBad)+2

isBad = np.array(isBad)
isGood = -isBad + 1
isGood = [i+1 for i,_ in enumerate(isGood) if _==1]
isBad = [i+1 for i,_ in enumerate(isBad) if _==1]

# calc all possible states
allStates = isGood.copy()
tmp = []
for i in range(rolls-1):
    for g1 in allStates:
        for g2 in isGood:
            tmp.append(g1+g2)
    allStates += tmp
allStates = list(set(allStates))
allStates.append(0)

# two actions are roll and quit
TransMatrix = np.zeros((2,nStates,nStates))
RewardMatrix = np.zeros((2,nStates,nStates))

TransRoll = TransMatrix[0]
TransQuit = TransMatrix[1]
RewardRoll = RewardMatrix[0]
RewardQuit = RewardMatrix[1]

for i, trans_row in enumerate(TransRoll):
    if i in allStates:
        for j in list(range(i+1,len(trans_row))):
            for good in isGood:
                if good == j-i:
                    RewardRoll[i,j] = good
                    TransRoll[i,j] = prob

        TransRoll[i,-1] = 1-sum(TransRoll[i,:-1])
        if i>= np.min(isGood):
            RewardRoll[i,-1] = -i
    else:
        TransRoll[i,-1] = 1
    TransQuit[i,-1] = 1



TransMatrix[0] = TransRoll
TransMatrix[1] = TransQuit
RewardMatrix[0] = RewardRoll
RewardMatrix[1] = RewardQuit
vi = mdptoolbox.mdp.ValueIteration(TransMatrix, RewardMatrix, 1)
vi.run()
np.max(vi.V)

7.3799805636540325

In [80]:
def expected_values(isBadSide):
    # get the good indicies
    good_indicies = []
    for i, isBad in enumerate(isBadSide):
        if not isBad:
            good_indicies.append(i + 1)
    print(good_indicies)
    
    N = len(isBadSide)
    max_states_n = 3 * N + 2

    T = np.zeros((2, max_states_n, max_states_n))
    
    T_roll = np.zeros((max_states_n, max_states_n))
    T_quit = np.zeros((max_states_n, max_states_n))
    
    R = np.zeros((2, max_states_n, max_states_n))
    
    R_roll = np.zeros((max_states_n, max_states_n))
    R_quit = np.zeros((max_states_n, max_states_n))
    
    possible_rows = [0] + good_indicies
    print(possible_rows)
    
    # build T and R matricies
    for row in range(0, max_states_n):
        # row vector
        T_roll_row = np.zeros(max_states_n)
        R_roll_row = np.zeros(max_states_n)
        
        if row in possible_rows:
            terminal_p = 1
            for idx in good_indicies:
                col = idx + row
                if col < max_states_n - 1:
                    T_roll_row[col] = 1 / N
                    terminal_p = terminal_p - (1 / N)
                    
                    R_roll_row[col] = idx
                    
                    if col not in possible_rows:
                        possible_rows.append(col)
            T_roll_row[max_states_n - 1] = terminal_p
            R_roll_row[max_states_n - 1] = -row
        else:
            T_roll_row[max_states_n - 1] = 1.0
        
        T_roll[row] = T_roll_row
        R_roll[row] = R_roll_row
        
        T_quit_row = np.zeros(max_states_n)
        T_quit_row[max_states_n - 1] = 1.0
        T_quit[row] = T_quit_row
        
    print(R_roll)
    T[0] = T_roll
    T[1] = T_quit
    R[0] = R_roll
    R[1] = R_quit
    
    vi = mdptoolbox.mdp.ValueIteration(T, R, 1.0)
    vi.run()
    
    optimal_policy = vi.policy
    expected_values = vi.V
    
    print(optimal_policy)
    print(expected_values)
    return R_roll, T_roll

In [81]:
r_roll, t_roll = expected_values([1,1,0,1,0,0])

[3, 5, 6]
[0, 3, 5, 6]
[[  0.   0.   0.   3.   0.   5.   6.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   3.   0.   5.   6.   0.   0.   0.   0.
    0.   0.   0.   0.   0.  -3.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   3.   0.   5.   6.   0.   0.
    0.   0.   0.   0.   0.  -5.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   3.   0.   5.   6.   0.
    0.   0.   0.   0.   0.  -6.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   3.   0.   5.
    6.   0.   0.   0.   0.  -8.]
 [  0.   0.   0.   0.   0.   0. 

In [82]:
t_roll

array([[0.        , 0.        , 0.        , 0.16666667, 0.        ,
        0.16666667, 0.16666667, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.16666667, 0.        , 0.16666667, 0.16666667,
        0.        , 0.        , 0.        , 0